# Conditional Neural Processes (CNP) for 1D regression.
[Conditional Neural Processes](https://arxiv.org/pdf/1807.01613.pdf) (CNPs) were
introduced as a continuation of
[Generative Query Networks](https://deepmind.com/blog/neural-scene-representation-and-rendering/)
(GQN) to extend its training regime to tasks beyond scene rendering, e.g. to
regression and classification.

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import datetime
import numpy as np
import torchsnooper
import plotting_utils_cnp as plotting
import data_generator as data
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import dask.dataframe as dd
import sys
sys.path.append('../utilities')
import utilities as utils
import import_ipynb
import conditional_neural_process_model as cnp

## Running Conditional Neural Processes

Now that we have defined the dataset as well as our model and its components we
can start building everything into the graph. Before we get started we need to
set some variables:

*   **`TRAINING_ITERATIONS`** - a scalar that describes the number of iterations
    for training. At each iteration we will sample a new batch of functions from
    the GP, pick some of the points on the curves as our context points **(x,
    y)<sub>C</sub>** and some points as our target points **(x,
    y)<sub>T</sub>**. We will predict the mean and variance at the target points
    given the context and use the log likelihood of the ground truth targets as
    our loss to update the model.
*   **`MAX_CONTEXT_POINTS`** - a scalar that sets the maximum number of contest
    points used during training. The number of context points will then be a
    value between 3 and `MAX_CONTEXT_POINTS` that is sampled at random for every
    iteration.
*   **`PLOT_AFTER`** - a scalar that regulates how often we plot the
    intermediate results.

In [ ]:
TRAINING_ITERATIONS = int(1401) # Total number of training points: training_iterations * batch_size * max_content_points
#BATCH_SIZE = 100 # number of simulation configurations

MAX_CONTEXT_POINTS = 1000 # 2000 # 4000
MAX_TARGET_POINTS =  2000 # 4000 # 8000
CONTEXT_IS_SUBSET = True
BATCH_SIZE = 1
CONFIG_WISE = False
PLOT_AFTER = int(200)
torch.manual_seed(0)

# all available x config/ physics parameters are ["radius","thickness","npanels","theta","length","height","z_offset","volume","nC_Ge77","time_0[ms]","x_0[m]","y_0[m]","z_0[m]","px_0[m]","py_0[m]","pz_0[m]","ekin_0[eV]","edep_0[eV]","time_t[ms]","x_t[m]","y_t[m]","z_t[m]","px_t[m]","py_t[m]","pz_t[m]","ekin_t[eV]","edep_t[eV]","nsec"]
# Comment: if using data version v1.1 for training, "radius","thickness","npanels","theta","length" is probably necessary
names_x=["radius","thickness","npanels","theta","length","r_0[m]","z_0[m]","time_t[ms]","r_t[m]","z_t[m]","L_t[m]","ln(E0vsET)","edep_t[eV]","nsec"]
name_y ='total_nC_Ge77[cts]'
x_size = len(names_x)
if isinstance(name_y,str):
    y_size = 1
else:
    y_size = len(name_y)

RATIO_TESTING_VS_TRAINING = 1/40
version="v1.3"
path_to_files=f"../simulation/out/LF/{version}/tier2/"
path_out = f'./out/'
f_out = f'{path_out}CNPGauss_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}'

In [ ]:
# Set data augmentation parameters
USE_DATA_AUGMENTATION = "mixup" #"smote" #False #"mixup"
USE_BETA = [0.1,0.1] # uniform => None, beta => [a,b] U-shape [0.1,0.1] Uniform [1.,1.] falling [0.2,0.5] rising [0.2,0.5]
SIGNAL_TO_BACKGROUND_RATIO = "" # "_1to4" # used for smote augmentation

if USE_DATA_AUGMENTATION:
    path_out = f'./out/{USE_DATA_AUGMENTATION}/'
    f_out = f'CNPGauss_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}_{USE_DATA_AUGMENTATION}{SIGNAL_TO_BACKGROUND_RATIO}'
    if USE_DATA_AUGMENTATION == "mixup":
        path_to_files = f"../simulation/out/LF/{version}/tier3/beta_{USE_BETA[0]}_{USE_BETA[1]}/"
        f_out = f'CNPGauss_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}_beta_{USE_BETA[0]}_{USE_BETA[1]}'
    elif USE_DATA_AUGMENTATION == "smote" and CONFIG_WISE == True:
        path_to_files = f"../simulation/out/LF/{version}/tier3/smote{SIGNAL_TO_BACKGROUND_RATIO}/"



In [ ]:
d_x, d_in, representation_size, d_out = x_size , x_size+y_size, 32, y_size+1
encoder_sizes = [d_in, 32, 64, 128, 128, 128, 64, 48, representation_size]
decoder_sizes = [representation_size + d_x, 32, 64, 128, 128, 128, 64, 48, d_out]

model = cnp.DeterministicModel(encoder_sizes,decoder_sizes)

model.load_state_dict(torch.load(f'./out/{f_out}_model.pth'))
model.eval()


In [ ]:
version="v1.3"
mode="LF"
filelist = utils.get_all_files(f"../simulation/out/{mode}/{version}/tier2/neutron")
num_total_points = 50000


MAX_CONTEXT_POINTS_NEW = int(1/3 * num_total_points)
MAX_TARGET_POINTS_NEW = 2 * (MAX_CONTEXT_POINTS_NEW)
bce = nn.BCELoss()

x_lf = np.empty([0,6])
sum_target_y_lf = np.empty([0,1])
mean_mu_cnp_lf = np.empty([0,1])
mean_sigma_cnp_lf = np.empty([0,1])
hist_target_sig = hist_target_bkg = hist_pred_sig = hist_pred_bkg = np.zeros(100)
fout = open(f'{path_out}{f_out}_training.txt', "a")

# create a PdfPages object
pdf = PdfPages(f'{path_out}{f_out}_result_{mode}.pdf')

for i,file in enumerate(filelist):
    
    path_to_files = file[:-4]
    dataset_config = data.DataGeneration(num_iterations=1, num_context_points=MAX_CONTEXT_POINTS_NEW, num_target_points=MAX_TARGET_POINTS_NEW, batch_size = 1, use_data_augmentation="None", path_to_files=path_to_files,x_size=x_size,y_size=y_size, mode = "config", ratio_testing=0.,names_x=names_x, name_y=name_y)
    data_config = dataset_config.get_data(0, CONTEXT_IS_SUBSET)
    # Get the predicted mean and variance at the target points for the testing set
    log_prob_config, mu_config, sigma_config = model(data_config.query, data_config.target_y)
    # Define the loss
    config_loss = -log_prob_config.mean()
    if max(mu_config[0].detach().numpy()) <= 1 and min(mu_config[0].detach().numpy()) >= 0:
            loss_bce_config = bce(mu_config,  data_config.target_y)
    else:
            loss_bce_config = -1.

    mu_config = mu_config[0].detach().numpy()
    target_y = data_config.target_y[0].detach().numpy()
    df = pd.read_csv(file, index_col=0)
    tmp = df[["fidelity","radius","thickness","npanels","theta","length"]].to_numpy()
    x_lf         = np.append(x_lf,[df[["fidelity","radius","thickness","npanels","theta","length"]].to_numpy()[0]],axis=0)

    sum_target_y_tmp = np.array([np.sum(target_y)])
    sum_target_y_lf    = np.append(sum_target_y_lf, [sum_target_y_tmp], axis=0)
    mean_mu_tmp = np.array([np.mean(mu_config)])
    mean_mu_cnp_lf = np.append(mean_mu_cnp_lf, [mean_mu_tmp], axis=0)
    mean_sigma_tmp = np.array([np.mean(sigma_config[0].detach().numpy())])
    mean_sigma_cnp_lf = np.append(mean_sigma_cnp_lf, [mean_sigma_tmp], axis=0)

    hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg = plotting.sum_hist(mu_config, target_y, hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
    print("{}/{} {}, {}, radius: {} cm, test loss: {}".format(i,len(filelist),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), mode,x_lf[-1,1], config_loss))
    fout.write("{}, Iteration: {}, test loss: {}\n".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), i, config_loss))

    fig = plotting.plot_result_configwise(mu_config, target_y, f'{config_loss:.2f}', x_lf[-1])
    pdf.savefig(fig)
    #plt.show()
    plt.clf()

    
fig1 = plotting.plot_result_summed(hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
pdf.savefig(fig1)
plt.show()
plt.clf()

fig2,_ = plt.subplots(3,2,sharey=True,figsize=(9, 9),layout="constrained")
ax2 = fig2.axes
for i in range(len(x_lf[0,:])-1):
    #ax2[i].plot(x_lf[:,0],mean_mu_cnp,'o')
    ax2[i].errorbar(x_lf[:,i+1], mean_mu_cnp_lf[:,0], yerr=mean_sigma_cnp_lf[:,0],fmt='o', color='teal')
    
    ax2[i].set_xlabel(names_x[i])
    if i % 2 == 0:
        ax2[i].set_ylabel(r'mean probability for nC on $^{76}Ge$')
    #ax2[i].set_ylim(0.0,0.15)
for i in range(len(x_lf[0,:])-1,len(ax2)):
     ax2[i].set_axis_off()
fig2.suptitle('Conditional Neutral Process', fontsize=10)
pdf.savefig(fig2)
plt.show()


fig3,_ = plt.subplots(3,2,sharey=True,figsize=(9, 9),layout="constrained")
ax3 = fig3.axes
for i in range(len(x_lf[0,:])-1):
    ax3[i].plot(x_lf[:,i+1],sum_target_y_lf,'o', color='teal')
    ax3[i].set_title('Simulation', fontsize=10)
    if i % 2 == 0:
        ax3[i].set_ylabel(r'nC on $^{76}Ge$')
    ax3[i].set_xlabel(names_x[i])
for i in range(len(x_lf[0,:])-1,len(ax3)):
     ax3[i].set_axis_off()
fig3.suptitle('Simulation', fontsize=10)
pdf.savefig(fig3)
plt.show()
pdf.close()

fout.close()

df = pd.DataFrame(x_lf, columns=["Mode","Radius[cm]","Thickness[cm]","NPanels","Theta[deg]","Length[cm]"])
df['Ge-77[nevents]'] = sum_target_y_lf
df['Ge-77_CNP'] = mean_mu_cnp_lf
df['Ge-77_CNP_err'] = mean_sigma_cnp_lf
df=df.round(decimals=4)
df.to_csv(f'{path_out}{f_out}_Ge77rates.csv')




In [ ]:

version_hf="v1.1"
mode="HF"
filelist = utils.get_all_files(f"../simulation/out/{mode}/{version_hf}/tier2/neutron")

x_hf = np.empty([0,6])
sum_target_y_hf = np.empty([0,1])
mean_mu_cnp_hf = np.empty([0,1])
mean_sigma_cnp_hf = np.empty([0,1])
upper_lim_hf = np.empty([0,1])
hist_target_sig = hist_target_bkg = hist_pred_sig = hist_pred_bkg = np.zeros(100)
fout = open(f'{path_out}{f_out}_training.txt', "a")

# create a PdfPages object
pdf=PdfPages(f'{path_out}{f_out}_result_{mode}.pdf')

for i,file in enumerate(filelist):

    path_to_files = file[:-4]
    num_total_points = 0
    with open(file, "rbU") as f:
        num_total_points += int(np.floor(sum(1 for _ in f)))

    MAX_CONTEXT_POINTS_NEW = int(1/3 * (num_total_points-1))
    MAX_TARGET_POINTS_NEW = 2 * MAX_CONTEXT_POINTS_NEW

    dataset_config = data.DataGeneration(num_iterations=1, num_context_points=MAX_CONTEXT_POINTS_NEW, num_target_points=MAX_TARGET_POINTS_NEW, batch_size = 1, use_data_augmentation="None", path_to_files=path_to_files,x_size=x_size,y_size=y_size, mode = "config", ratio_testing=0.,names_x=names_x, name_y=name_y)
    data_config = dataset_config.get_data(0, CONTEXT_IS_SUBSET)
    
    # Get the predicted mean and variance at the target points for the testing set
    log_prob_config, mu_config, sigma_config = model(data_config.query, data_config.target_y)
    # Define the loss
    config_loss = -log_prob_config.mean()
    if max(mu_config[0].detach().numpy()) <= 1 and min(mu_config[0].detach().numpy()) >= 0:
            loss_bce_config = bce(mu_config,  data_config.target_y)
    else:
            loss_bce_config = -1.

    mu_config = mu_config[0].detach().numpy()
    
    target_y = data_config.target_y[0].detach().numpy()
    df = pd.read_csv(file, index_col=0)
    x_hf         = np.append(x_hf,[df[["fidelity","radius","thickness","npanels","theta","length"]].to_numpy()[0]],axis=0)
    #x         = np.append(x,[data_config.query[1][0][0][-5:].numpy()],axis=0)
    sum_target_y_tmp = np.array([np.sum(target_y)])
    sum_target_y_hf    = np.append(sum_target_y_hf, [sum_target_y_tmp], axis=0)
    mean_mu_tmp = np.array([np.mean(mu_config)])
    upper_lim_hf = np.append(upper_lim_hf,[np.array([np.percentile(mu_config,95.)])], axis=0)
    mean_mu_cnp_hf = np.append(mean_mu_cnp_hf, [mean_mu_tmp], axis=0)
    mean_sigma_tmp = np.array([np.mean(sigma_config[0].detach().numpy())])
    mean_sigma_cnp_hf = np.append(mean_sigma_cnp_hf, [mean_sigma_tmp], axis=0)

    hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg = plotting.sum_hist(mu_config, target_y, hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
    print("{}/{} {}, {}, radius: {} cm, test loss: {}".format(i,len(filelist),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), mode,x_hf[-1,1], config_loss))
    fout.write("{}, Iteration: {}, test loss: {}\n".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), i, config_loss))
    
    fig = plotting.plot_result_configwise(mu_config, target_y, f'{config_loss:.2f}', x_hf[-1])
    plt.close(fig)
    pdf.savefig(fig)
    #plt.show()
    plt.clf()

    
fig1 = plotting.plot_result_summed(hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
pdf.savefig(fig1)
plt.show()
plt.clf()

fig2,_ = plt.subplots(3,2,sharey=True,figsize=(9, 9),layout="constrained")
ax2 = fig2.axes
for i in range(len(x_hf[0,:])-1):
    #ax2[i].plot(x_lf[:,0],mean_mu_cnp,'o')
    ax2[i].errorbar(x_hf[:,i+1], mean_mu_cnp_hf[:,0], yerr=mean_sigma_cnp_hf[:,0],fmt='o', color='teal')
    
    ax2[i].set_xlabel(names_x[i])
    if i % 2 == 0:
        ax2[i].set_ylabel(r'mean probability for nC on $^{76}Ge$')
for i in range(len(x_hf[0,:])-1,len(ax2)):
     ax2[i].set_axis_off()
fig2.suptitle('Conditional Neutral Process', fontsize=10)
pdf.savefig(fig2)
plt.show()

fig3,_ = plt.subplots(3,2,sharey=True,figsize=(9, 9),layout="constrained")
ax3 = fig3.axes
for i in range(len(x_hf[0,:])-1):
    ax3[i].plot(x_hf[:,i+1],sum_target_y_hf,'o', color='teal')
    if i % 2 == 0:
        ax3[i].set_ylabel(r'nC on $^{76}Ge$')
    ax3[i].set_xlabel(names_x[i])
for i in range(len(x_hf[0,:])-1,len(ax3)):
     ax3[i].set_axis_off()
pdf.savefig(fig3)
plt.show()

pdf.close()
fout.close()

df= pd.read_csv(f'{path_out}{f_out}_Ge77rates.csv', index_col=0)
x = df[["Mode","Radius[cm]","Thickness[cm]","NPanels","Theta[deg]","Length[cm]","Ge-77[nevents]","Ge-77_CNP","Ge-77_CNP_err"]].to_numpy()
x_tmp = np.append(x_hf, sum_target_y_hf, axis=1)
x_tmp = np.append(x_tmp, mean_mu_cnp_hf, axis=1)
x_tmp = np.append(x_tmp, mean_sigma_cnp_hf, axis=1)
x = np.append(x, x_tmp, axis=0)
df = pd.DataFrame(x, columns=df.columns)

df=df.round(decimals=4)
df.to_csv(f'{path_out}{f_out}_Ge77rates.csv')
df.to_csv(f'../multi-fidelity-gaussian-process/in/Ge77_rates_CNP_{version}.csv')


In [ ]:
fig4,_ = plt.subplots(3,2,sharey=True,figsize=(9, 9),layout="constrained")
fig4.suptitle('Conditional Neutral Process', fontsize=10)
ax4 = fig4.axes

for i in range(len(x_lf[0,:])-1):
    ax4[i].errorbar(x_lf[:,i+1], mean_mu_cnp_lf[:,0], yerr=mean_sigma_cnp_lf[:,0],fmt='o', color='teal')
    ax4[i].set_xlabel(names_x[i])
    if i % 2 == 0:
        ax4[i].set_ylabel(r'mean probability for nC on $^{76}Ge$')

for i in range(len(x_hf[0,:])-1):
    #ax2[i].plot(x_lf[:,0],mean_mu_cnp,'o')
    ax4[i].errorbar(x_hf[:,i+1], mean_mu_cnp_hf[:,0], yerr=mean_sigma_cnp_hf[:,0],fmt='o', color='orangered')
    ax4[i].set_xlabel(names_x[i])

for i in range(len(x_lf[0,:])-1,len(ax4)):
     ax4[i].set_axis_off()



for i in range(len(x_hf[0,:])-1,len(ax4)):
     ax4[i].set_axis_off()

#pdf.savefig(fig4)
plt.show()